In [1]:
#!/usr/bin/env python3
import numpy as np

from utils import Data
from deepPC_acados import DeePC   # your Acados/CasADi DeePC implementation

# 1) Simulate a simple 2-state LTI system: x_{k+1} = A x_k + B u_k,  y_k = C x_k
A = np.array([[1.0, 1.0],
              [0.0, 1.0]])
B = np.array([[0.5],
              [1.0]])
C = np.eye(2)

T_data = 60
x = np.zeros(2)
u_data, y_data = [], []

for _ in range(T_data):
    u_k = np.random.uniform(-1, 1)
    x = A.dot(x) + (B.flatten() * u_k)
    y = C.dot(x)
    u_data.append([u_k])
    y_data.append(y)

u_data = np.vstack(u_data)   # shape (T_data × 1)
y_data = np.vstack(y_data)   # shape (T_data × 2)

data = Data(u=u_data, y=y_data)

# 2) Instantiate DeePC-Acados with Tini=10, horizon=15
Tini, horizon = 10, 15
deepc = DeePC(data, Tini=Tini, horizon=horizon, explained_variance=None)

# 3) Configure the OCP inside Acados
#    We’ll use a simple tracking cost:   Σ‖y‖²_Q  +  Σ‖u‖²_R
Qy = np.diag([10.0, 10.0])   # weight on each output channel
Ru = 1.0                     # weight on input effort
umin, umax = -1.0, 1.0       # box constraints on u

deepc.build_ocp(
    Q_y     = Qy, 
    R_u     = Ru,
    umin    = umin,
    umax    = umax,
    qp_solver               = "FULL_CONDENSING_QPOASES",
    nlp_solver_type         = "SQP",
    nlp_solver_max_iter     = 50,
    qp_solver_cond_N        = horizon
)

# 4) Prepare the “initial‐condition” data block
#    (first Tini samples become u_ini, y_ini)
data_ini = Data(
    u = u_data[:Tini, :],
    y = y_data[:Tini, :]
)

# 5) Solve the DeePC problem via Acados
u_opt, info = deepc.solve(data_ini)

print("Optimal control sequence (horizon × 1):")
print(u_opt.flatten())

print(f"\nAcados solver status: {info['solver_status']}")
print(f"Optimal objective:   {info['cost']:.4f}")


ModuleNotFoundError: No module named 'utils'

In [2]:
import os
import time
from datetime import datetime
import threading
from pathlib import Path

import numpy as np
import scipy.io as sio
import pandas as pd

import Jetson.GPIO as GPIO
GPIO.cleanup()

import board
import busio
from adafruit_pca9685 import PCA9685

import cantools
import can
import casadi as ca
from acados_template import AcadosOcp, AcadosOcpSolver
import deepctools as dpc

from utils import *

# ─── LOAD DATA FOR DeePC ───────────────────────────────────────────────
xlsx_path="PWMDynoSpeedNewsMCT.xlsx"
seq_len = 50
ds = MultiSheetTimeSeriesDataset(xlsx_path, seq_len, normalize=True, cache_path="save/timeseries_dataset0527.npz")




Start loading data...
DataSheetName: 0521-sMCT1,  pwm shape: (76509,) vel shape: (76509,)
DataSheetName: 0521-sMCT2,  pwm shape: (76537,) vel shape: (76537,)
DataSheetName: 0521-sMCT3,  pwm shape: (76464,) vel shape: (76464,)
[MultiSheetTimeSeriesDataset] X shape: (229360, 50), y shape: (229360, 50)
Saved cache → save/timeseries_dataset0527.npz


In [ ]:
import numpy as np
from casadi import MX, vertcat
from acados_template import AcadosQp, AcadosQpSolver

# === 1) Offline: load data and build Hankel =========================
ud = np.load("u_data.npy")   # shape (m, T)
yd = np.load("y_data.npy")   # shape (p, T)
m, T = ud.shape
p, _ = yd.shape
T_ini = 10                   # your choice
N     = 20                   # prediction horizon
Lg    = T - (T_ini + N) + 1

def hankel(seq, rows, cols):
    # Build block-Hankel with `rows` block rows and `cols` columns
    return np.vstack([seq[:, i:i+cols] for i in range(rows)])

H_u = hankel(ud, T_ini+N, Lg)   # shape (m*(T_ini+N), Lg)
H_y = hankel(yd, T_ini+N, Lg)   # shape (p*(T_ini+N), Lg)
U_p = H_u[        :m*T_ini, :]  # shape (m*T_ini, Lg)
U_f = H_u[m*T_ini: m*(T_ini+N),:]
Y_p = H_y[        :p*T_ini, :]  # shape (p*T_ini, Lg)
Y_f = H_y[p*T_ini: p*(T_ini+N),:]

# === 2) Build QP matrices H, f =======================================
nz = Lg + m*N + p*N
# Hessian
H = np.zeros((nz, nz))
# R-blocks on u
for k in range(N):
    i = Lg + k*m
    H[i:i+m, i:i+m] = np.eye(m) * 1.0   # replace with your R
# Q-blocks on y
for k in range(N):
    j = Lg + m*N + k*p
    H[j:j+p, j:j+p] = np.eye(p) * 10.0  # replace with your Q

# gradient placeholder (we'll update for each r)
f = np.zeros(nz)

# === 3) Equality constraints A_eq z = b_eq ===========================
# rows = m*T_ini + p*T_ini + m*N + p*N
A_eq = np.zeros((m*T_ini + p*T_ini + m*N + p*N, nz))
b_eq = np.zeros((A_eq.shape[0],))

# Up * g = u_ini
A_eq[0:m*T_ini, 0: Lg]       = U_p
# Yp * g = y_ini
A_eq[m*T_ini: m*T_ini+p*T_ini, 0: Lg] = Y_p
# Uf * g - u = 0
A_eq[m*T_ini+p*T_ini: m*T_ini+p*T_ini+m*N, 0: Lg] = U_f
A_eq[m*T_ini+p*T_ini: m*T_ini+p*T_ini+m*N, Lg: Lg+m*N] = -np.eye(m*N)
# Yf * g - y = 0
i0 = m*T_ini+p*T_ini+m*N
A_eq[i0: i0+p*N, 0: Lg] = Y_f
A_eq[i0: i0+p*N, Lg+m*N: Lg+m*N+p*N] = -np.eye(p*N)

# === 4) Inequality bounds on u, y ================================
# let's say u_min <= u <= u_max, same for y
u_min, u_max = -1.0, 1.0
y_min, y_max = -5.0, 5.0

# selection matrix for u
S_u = np.zeros((m*N, nz))
for k in range(N):
    S_u[k*m:(k+1)*m, Lg + k*m: Lg+(k+1)*m] = np.eye(m)
# selection for y
S_y = np.zeros((p*N, nz))
for k in range(N):
    S_y[k*p:(k+1)*p, Lg + m*N + k*p: Lg + m*N + (k+1)*p] = np.eye(p)

# stack ineq: [ S_u; -S_u; S_y; -S_y ] * z <= [u_max; -u_min; y_max; -y_min]
A_ineq = np.vstack([S_u, -S_u, S_y, -S_y])
l_ineq = np.hstack([u_min*np.ones(m*N),
                    -u_max*np.ones(m*N),
                    y_min*np.ones(p*N),
                    -y_max*np.ones(p*N)])
u_ineq = np.hstack([u_max*np.ones(m*N),
                    -u_min*np.ones(m*N),
                    y_max*np.ones(p*N),
                    -y_min*np.ones(p*N)])

# === 5) Create and initialize acados QP solver =====================
qp = AcadosQp()
qp.solver_options.qp_solver = 'FULL_CONDENSING_HPIPM'
qp.dims.n       = nz
qp.dims.n_eq    = A_eq.shape[0]
qp.dims.n_ineq  = A_ineq.shape[0]

qp.cost.H = H
qp.cost.g = f        # will update for each r
qp.cost.idx_z = np.arange(nz)

qp.constraints.A = np.vstack([A_eq, A_ineq])
# equality rows are first
qp.constraints.lh = np.hstack([b_eq, l_ineq])
qp.constraints.uh = np.hstack([b_eq, u_ineq])

# build solver
solver = AcadosQpSolver(qp)

# === 6) At each control step: update parameters and solve ===========
def control_step(u_ini_vals, y_ini_vals, r_vals):
    # 1) update b_eq[:m*T_ini] = u_ini, next p*T_ini = y_ini
    b_eq[:m*T_ini]             = u_ini_vals.flatten()
    b_eq[m*T_ini:m*T_ini+p*T_ini] = y_ini_vals.flatten()
    solver.set('constraints.lh', np.hstack([b_eq, l_ineq]))
    solver.set('constraints.uh', np.hstack([b_eq, u_ineq]))

    # 2) update f = -2 * [zeros;    zeros;  Q*(r); ... repeated] 
    #    since J = ... + (y-r)'Q(y-r) = y'Qy - 2r'Qy + const.
    f[:] = 0
    for k in range(N):
        idx = Lg + m*N + k*p
        f[idx:idx+p] = -2 * (qp.cost.H[idx:idx+p, idx:idx+p] @ r_vals[k*p:(k+1)*p])
    solver.set('cost.g', f)

    # 3) solve
    status = solver.solve()
    z_opt = solver.get('solution')
    u_opt = z_opt[Lg: Lg+m*N]
    # apply only first m entries
    return u_opt[:m]

# === Example usage ===============================================
# suppose you have buffered the last T_ini steps into arrays
u_ini_meas = np.zeros((m, T_ini))
y_ini_meas = np.zeros((p, T_ini))
# and a reference trajectory of length N
r_traj      = np.tile(np.array([[1.0]]), (p, N))  # shape (p, N) flattened

u0 = control_step(u_ini_meas, y_ini_meas, r_traj.flatten())
print("apply u0 =", u0)
